In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuzhang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
test = pd.read_csv('/Users/yuzhang/Downloads/data/test.csv')
train = pd.read_csv('/Users/yuzhang/Downloads/data/train.csv')

In [4]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
X = train.drop(['id'], axis = 1)
X.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
key = X["keyword"].value_counts().index[0]
print(key)
loc = X["location"].value_counts().index[0]
print(loc)

fatalities
USA


In [8]:
train['keyword']=train['keyword'].fillna(train['keyword'].value_counts().index.max())
train['location']=train['location'].fillna(train['location'].value_counts().index.max())
train.head()

,id,keyword,location,text,target
0,1,wrecked,åø\_(?)_/åø,Our Deeds are the Reason of this #earthquake M...,1
1,4,wrecked,åø\_(?)_/åø,Forest fire near La Ronge Sask. Canada,1
2,5,wrecked,åø\_(?)_/åø,All residents asked to 'shelter in place' are ...,1
3,6,wrecked,åø\_(?)_/åø,"13,000 people receive #wildfires evacuation or...",1
4,7,wrecked,åø\_(?)_/åø,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
test['keyword'] = test['keyword'].fillna(test['keyword'].value_counts().idxmax())
test['location'] = test['location'].fillna(test['location'].value_counts().idxmax())
test.isnull().sum()

id          0
keyword     0
location    0
text        0
dtype: int64

In [13]:
!pip install -U textblob

     |████████████████████████████████| 636 kB 4.5 MB/s            


In [14]:
X=train.iloc[:,1:4]
y=train.iloc[:,4:]
X["text"]=X["text"].apply(lambda x:" ".join(x.lower() for x in x.split())) # Lower words
X["text"]=X["text"].str.replace("[^\w\s]"," ") # Clear ".,!#"
X["text"]=X["text"].str.replace("\d"," ") # Clear Number
X["text"]=X["text"].str.replace("https"," ") # Clear https

# Stopwords
sw=stopwords.words("english")
X["text"]=X["text"].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
# Lemmi
from textblob import Word
nltk.download("wordnet")
X["text"]=X["text"].apply(lambda x:" ".join([Word(x).lemmatize() for x in x.split()]))
X["text"][:5]

<ipython-input-14-79288de01214>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  X["text"]=X["text"].str.replace("[^\w\s]"," ") # Clear ".,!#"
<ipython-input-14-79288de01214>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  X["text"]=X["text"].str.replace("\d"," ") # Clear Number
[nltk_data] Downloading package wordnet to /Users/yuzhang/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0           deed reason earthquake may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3    people receive wildfire evacuation order calif...
4    got sent photo ruby alaska smoke wildfire pour...
Name: text, dtype: object

In [16]:
# For Test Data
test_x=test.iloc[:,1:4]
test_y=test.iloc[:,4:]

test_x["text"]=test_x["text"].apply(lambda x:" ".join(x.lower() for x in x.split() ))
test_x["text"]=test_x["text"].str.replace("[^\w\s]"," ")
test_x["text"]=test_x["text"].str.replace("\d"," ")
test_x["text"]=test_x["text"].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
test_x["text"]=test_x["text"].str.replace("https"," ") # Clear https

#Lemmi
test_x["text"]=test_x["text"].apply(lambda x:" ".join([Word(x).lemmatize() for x in x.split()]))
test_x["text"][:5]

<ipython-input-16-e3bf4965f624>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  test_x["text"]=test_x["text"].str.replace("[^\w\s]"," ")
<ipython-input-16-e3bf4965f624>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test_x["text"]=test_x["text"].str.replace("\d"," ")


0                          happened terrible car crash
1    heard earthquake different city stay safe ever...
2    forest fire spot pond goose fleeing across str...
3                 apocalypse lighting spokane wildfire
4                   typhoon soudelor kill china taiwan
Name: text, dtype: object

In [18]:
X['sentence']=X['keyword']+" "+X["text"]
train_text = X["sentence"].values
print(train_text)

['wrecked deed reason earthquake may allah forgive u'
 'wrecked forest fire near la ronge sask canada'
 'wrecked resident asked shelter place notified officer evacuation shelter place order expected'
 ... 'wrecked utc km volcano hawaii http co zdtoyd ebj'
 'wrecked police investigating e bike collided car little portugal e bike rider suffered serious non life threatening injury'
 'wrecked latest home razed northern california wildfire abc news http co ymy rskq']


In [25]:
test_x['sentence']=test_x['keyword']+" "+test_x["text"]
test_text = test_x["sentence"].values
print(test_text)

['deluged happened terrible car crash'
 'deluged heard earthquake different city stay safe everyone'
 'deluged forest fire spot pond goose fleeing across street cannot save'
 ... 'deluged green line derailment chicago http co utbxlcbiuy'
 'deluged meg issue hazardous weather outlook hwo http co x rbqjhn'
 'deluged cityofcalgary activated municipal emergency plan yycstorm']


In [26]:
x_train, x_test, y_train, y_test=train_test_split(train_text, y, test_size =0.2, random_state=1)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 1500, min_df = 5, max_df = 0.7)
vectorizer.fit(x_train)
print(x_train)
x_train_tfidf=vectorizer.transform(x_train).toarray()
x_test_tfidf=vectorizer.transform(x_test).toarray()
test_last=vectorizer.transform(test_text).toarray()

['bridge%20collapse sioux city fire official believe bridge collapse lead cement truck roll siouxland matter siouxlan http co sztgmbkohg'
 'military strongly support military amp family cock sucker dc work'
 'trouble lucysforsale funny cause dumb as young one get n trouble lol'
 ...
 'flooding monsoon flooding kill dozen myanmar prompting call help http co r vpaklhvi'
 'annihilated completely annihilated cech paul keegan time alive'
 'obliterate whereas jez obliterate national debt give lot new benefit simply printing money genius co reffbkvg r']


In [28]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

bn=BernoulliNB(alpha=0.2)
bn.fit(x_train_tfidf, y_train)
y_pred=bn.predict(x_test_tfidf)
cm=confusion_matrix(y_test, y_pred)
print(cm)
f1=f1_score(y_test, y_pred)
print(f1)
cr=classification_report(y_test, y_pred)
print(cr)
accuracy=cross_val_score(bn,x_test_tfidf, y_test, cv=10).mean()
print(accuracy)

/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[[772 110]
 [189 452]]
0.7514546965918537
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       882
           1       0.80      0.71      0.75       641

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/yuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please cha

0.7662839697282422


In [31]:
prediction1=bn.predict(test_last)
data={'id':test["id"], 'target':prediction1}
output=pd.DataFrame(data, columns=["id", 'target'])
output.index=test.index

output.to_csv("submission.csv", index = False)

a = pd.read_csv("submission.csv")
a

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


#### Using CATBOOST